In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
# load ascii text 
filename = "my_words.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()

# OR Write your own:

# covert to lowercase
raw_text = raw_text.lower()

In [5]:
raw_text

"hollywood's bleeding, vampires feedin'\ndarkness turns to dust\neveryone's gone, but no one's leavin'\nnobody left but us\ntryna\u2005chase\u2005a feelin', but\u2005we'll never feel it\nridin' on the\u2005last train home\ndyin' in our sleep, we're living out a dream\nwe only make it out alone\ni just keep on hopin' that you call me\nyou say you wanna see me, but you can't right now\nyou never took the time to get to know me\nwas scared of losin' somethin' that we never found\nwe're running out of reasons, but we can't let go\nyeah, hollywood is bleeding, but we call it home\noutside, the winter sky turnin' grey\ncity up in smoke, it's only ash when it rains\nhowl at the moon and go to sleep in the day\nlove for everybody 'til the drugs fade away\nin the mornin', blocking out the sun with the shades\nshe gotta check her pulse and tell herself that she okay\nit seem like dying young is an honor\nbut who'd be at my funeral? i wonder\ni go out, and all they eyes on me\ni show out, do you 

In [6]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
char_to_int

{'\n': 0,
 ' ': 1,
 "'": 2,
 ',': 3,
 '-': 4,
 '?': 5,
 'a': 6,
 'b': 7,
 'c': 8,
 'd': 9,
 'e': 10,
 'f': 11,
 'g': 12,
 'h': 13,
 'i': 14,
 'j': 15,
 'k': 16,
 'l': 17,
 'm': 18,
 'n': 19,
 'o': 20,
 'p': 21,
 'r': 22,
 's': 23,
 't': 24,
 'u': 25,
 'v': 26,
 'w': 27,
 'y': 28,
 '\u2005': 29}

In [8]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  1485
Total Vocab:  30


In [9]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  1385


In [10]:
print(len(dataX))
print(len(dataX[0]))

1385
100


In [11]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
print(X.shape)
print(y.shape)

(1385, 100, 1)
(1385, 30)


In [13]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W1112 00:56:12.800593 140688742917888 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1112 00:56:12.830976 140688742917888 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1112 00:56:12.834806 140688742917888 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1112 00:56:13.189406 140688742917888 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1112 00:56:13.198332 

In [14]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [15]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

W1112 00:56:17.415277 140688742917888 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/20
1385/1385 [==============================] - 5s 4ms/step - loss: 3.1952

Epoch 00001: loss improved from inf to 3.19518, saving model to weights-improvement-01-3.1952.hdf5
Epoch 2/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9964

Epoch 00002: loss improved from 3.19518 to 2.99642, saving model to weights-improvement-02-2.9964.hdf5
Epoch 3/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9800

Epoch 00003: loss improved from 2.99642 to 2.98005, saving model to weights-improvement-03-2.9800.hdf5
Epoch 4/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9720

Epoch 00004: loss improved from 2.98005 to 2.97200, saving model to weights-improvement-04-2.9720.hdf5
Epoch 5/20
1385/1385 [==============================] - 4s 3ms/step - loss: 2.9676

Epoch 00005: loss improved from 2.97200 to 2.96758, saving model to weights-improvement-05-2.9676.hdf5
Epoch 6/20
1385/1385 [==============================] - 5s 3ms/step -

In [16]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
"  out, and all they eyes on me
i show out, do you like what you see?
and now they closin' in on me
le "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         